In [42]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

In [43]:
data = pd.read_csv('data/day08_digit recognizor.csv')
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
X,y = data.iloc[:,1:],data.iloc[:,0]

selector = VarianceThreshold()
X_var0 = selector.fit_transform(X)
X_var0 = pd.DataFrame(X_var0)
X_var0.head()

,0,1,2,3,4,5,6,7,8,9,...,698,699,700,701,702,703,704,705,706,707
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
np.median(X.var().values) 

1352.286703180131

In [46]:
X_fsvar = VarianceThreshold(threshold= np.median(X.var().values))
X_fsvar = X_fsvar.fit_transform(X)
X_fsvar.shape

(42000, 392)

In [47]:
rfc = RandomForestClassifier(n_estimators=100,random_state=100)
score = cross_val_score(rfc,X_fsvar,y,cv=3).mean()
score

0.9619761904761904

In [38]:
from sklearn.feature_selection import SelectKBest,chi2

#  相关性
X_fschi = SelectKBest(chi2,k=300).fit_transform(X_fsvar,y)
X_fschi = pd.DataFrame(X_fschi)
X_fschi

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,18,30,137,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,25,130,155,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,46,0,0,0,0,0,0,0,0,0
41996,0,0,0,0,0,0,0,0,0,0,...,75,0,0,0,0,0,0,0,0,0
41997,0,0,0,0,0,0,0,0,0,0,...,255,0,128,255,255,255,128,0,0,0
41998,0,0,0,146,237,252,189,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
rfc = RandomForestClassifier(n_estimators=100,random_state=100)
score = cross_val_score(rfc,X_fschi,y,cv=3).mean()
score

0.9569761904761904

In [51]:
# 1. 卡方检验
chivalue,pvalue_chi = chi2(X_fsvar,y)

In [54]:
k = chivalue.shape[0] - (pvalue_chi > 0.05).sum()
k

392

In [57]:
X_fschi = SelectKBest(chi2, k = k).fit_transform(X_fsvar,y)
X_fschi

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,  74, 250,  59]], dtype=int64)

In [59]:
rfc = RandomForestClassifier(n_estimators=100,random_state=100)
score = cross_val_score(rfc,X_fschi,y,cv=3).mean()
score

0.9619761904761904

In [61]:
#  2. F 检验
from sklearn.feature_selection import f_classif
F,pvaules_f = f_classif(X_fsvar,y)
k = F.shape[0] - (pvaules_f > 0.05).sum()
k

392

In [63]:
# 3. 互信息法  捕捉线性关系与非线性关系
from sklearn.feature_selection import mutual_info_classif

result = mutual_info_classif(X_fsvar,y)
result

array([0.07070842, 0.08989007, 0.10374705, 0.1085555 , 0.11374049,
       0.10457419, 0.07837875, 0.04708195, 0.07906694, 0.09492777,
       0.12738845, 0.13961527, 0.16044077, 0.16459462, 0.15249001,
       0.12904815, 0.09255529, 0.06613671, 0.03376724, 0.02686833,
       0.07007679, 0.09567284, 0.1242633 , 0.14869632, 0.1832557 ,
       0.2077109 , 0.22274643, 0.23971838, 0.21915073, 0.17976752,
       0.13650667, 0.10431588, 0.07829957, 0.06403252, 0.04857968,
       0.02745759, 0.05480852, 0.07489086, 0.10062084, 0.11649414,
       0.13762899, 0.14803382, 0.16135979, 0.17407228, 0.18704728,
       0.17801064, 0.14741208, 0.13302724, 0.11279843, 0.09339667,
       0.08223652, 0.0753958 , 0.07089743, 0.0442542 , 0.06161143,
       0.0754478 , 0.08494614, 0.09672117, 0.10697825, 0.10424115,
       0.10661244, 0.11305829, 0.11750648, 0.11431121, 0.11632763,
       0.11923433, 0.11474979, 0.10118663, 0.09594978, 0.08175721,
       0.07307297, 0.04163769, 0.05784922, 0.06189211, 0.07597

In [66]:
k = result.shape[0] - sum(result <= 0)
k

392